# HR Data -  Pay Rate prediction

In [48]:
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support as score

from sklearn.metrics import explained_variance_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score

from time import time

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression, RANSACRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.svm import SVR
from sklearn.svm import LinearSVR
from sklearn.linear_model import Lasso, Ridge


from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR


import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sb
import numpy as np
import pandas as pd

In [27]:
hr = pd.read_csv('D:\OSSI\Whitepapers\Cloud_Learning\Visualization_Cloud_Service\Analysis_Examples\Kaggle_HR_Dataset\core_dataset.csv')
hr.columns = ['Name','EmpNum','State','Zip','Date_of_birth','Age','Sex','MaritalDesc','CitizenDesc','Hispanic/Latino','RaceDesc','Date_of_Hire','Date_of_Termination','Reason_For_Termination','Employment_Status','Department','Position','Pay_Rate','Manager_Name','Employment_Source','Performance_Score']
hr = hr.drop(hr.index[301])
print(hr)

                      Name        EmpNum State      Zip Date_of_birth   Age  \
0               Brown, Mia  1.103024e+09    MA   1450.0    11/24/1985  32.0   
1     LaRotonda, William    1.106027e+09    MA   1460.0     4/26/1984  33.0   
2         Steans, Tyrone    1.302053e+09    MA   2703.0      9/1/1986  31.0   
3          Howard, Estelle  1.211051e+09    MA   2170.0     9/16/1985  32.0   
4              Singh, Nan   1.307060e+09    MA   2330.0     5/19/1988  29.0   
5         Smith, Leigh Ann  7.110077e+08    MA   1844.0     6/14/1987  30.0   
6      LeBlanc, Brandon  R  1.102024e+09    MA   1460.0     6/10/1984  33.0   
7              Quinn, Sean  1.206043e+09    MA   2045.0     11/6/1984  33.0   
8        Boutwell, Bonalyn  1.307060e+09    MA   2468.0      4/4/1987  30.0   
9        Foster-Baker, Amy  1.201031e+09    MA   2050.0     4/16/1979  38.0   
10             King, Janet  1.001495e+09    MA   1902.0     9/21/1954  63.0   
11        Zamora, Jennifer  1.112031e+09    MA   206

In [28]:
hr_no_dot = hr.drop('Date_of_Termination',axis=1)

In [29]:
le = preprocessing.LabelEncoder()
hr_no_dot = hr_no_dot.apply(le.fit_transform)
hr_no_dot

,Name,EmpNum,State,Zip,Date_of_birth,Age,Sex,MaritalDesc,CitizenDesc,Hispanic/Latino,RaceDesc,Date_of_Hire,Reason_For_Termination,Employment_Status,Department,Position,Pay_Rate,Manager_Name,Employment_Source,Performance_Score
0,28,82,10,3,51,7,0,1,2,0,2,17,2,0,0,0,35,3,3,3
1,149,102,10,4,143,8,1,0,2,0,2,8,2,0,0,0,22,3,20,3
2,262,182,10,119,266,6,1,3,2,0,5,88,2,0,0,0,38,3,7,3
3,126,168,10,86,274,7,0,1,2,0,5,27,2,0,0,1,19,3,15,4
4,249,200,10,98,164,4,0,3,2,0,5,46,2,0,0,1,6,3,20,4
5,253,4,10,25,198,5,0,1,2,0,1,86,4,4,0,1,16,3,3,3
6,155,76,10,4,194,8,1,1,2,0,5,7,2,0,0,18,73,9,9,3
7,221,158,10,47,58,8,1,1,0,0,2,31,4,4,0,18,73,9,3,3
8,24,206,10,112,148,5,0,1,2,0,1,27,2,0,0,21,43,3,3,0
9,88,138,10,49,133,13,0,1,2,2,5,4,2,0,0,21,43,2,13,3


In [30]:
corr = hr_no_dot.corr()
print(corr)

                            Name    EmpNum     State       Zip  Date_of_birth  \
Name                    1.000000 -0.062629 -0.068161  0.063956      -0.021010   
EmpNum                 -0.062629  1.000000  0.034308  0.091633       0.012885   
State                  -0.068161  0.034308  1.000000  0.200278       0.027510   
Zip                     0.063956  0.091633  0.200278  1.000000       0.034015   
Date_of_birth          -0.021010  0.012885  0.027510  0.034015       1.000000   
Age                     0.013301  0.170957  0.015939  0.061255      -0.139363   
Sex                     0.030142  0.020165  0.013284 -0.003618      -0.035758   
MaritalDesc            -0.043863 -0.073187  0.005485  0.091319      -0.009466   
CitizenDesc            -0.082492 -0.041620 -0.120845 -0.040820      -0.013275   
Hispanic/Latino        -0.042185  0.015645  0.001683  0.013738      -0.041885   
RaceDesc               -0.058819 -0.028978 -0.063482  0.065803       0.089591   
Date_of_Hire           -0.01

In [31]:
hr_no_dot.columns

Index(['Name', 'EmpNum', 'State', 'Zip', 'Date_of_birth', 'Age', 'Sex',
       'MaritalDesc', 'CitizenDesc', 'Hispanic/Latino', 'RaceDesc',
       'Date_of_Hire', 'Reason_For_Termination', 'Employment_Status',
       'Department', 'Position', 'Pay_Rate', 'Manager_Name',
       'Employment_Source', 'Performance_Score'],
      dtype='object')

In [34]:
X = hr_no_dot.loc[:,['State', 'Zip', 'Date_of_birth', 'Age', 'Sex',
       'MaritalDesc', 'CitizenDesc', 'Hispanic/Latino', 'RaceDesc',
       'Date_of_Hire', 'Reason_For_Termination', 'Employment_Status',
       'Department', 'Position', 'Manager_Name',
       'Employment_Source', 'Performance_Score']]
y = hr_no_dot.loc[:,['Pay_Rate']]

print(X)
print(y)

     State  Zip  Date_of_birth  Age  Sex  MaritalDesc  CitizenDesc  \
0       10    3             51    7    0            1            2   
1       10    4            143    8    1            0            2   
2       10  119            266    6    1            3            2   
3       10   86            274    7    0            1            2   
4       10   98            164    4    0            3            2   
5       10   25            198    5    0            1            2   
6       10    4            194    8    1            1            2   
7       10   47             58    8    1            1            0   
8       10  112            148    5    0            1            2   
9       10   49            133   13    0            1            2   
10      10   35            278   34    0            1            2   
11      10   53            261   13    0            3            2   
12      10   41            147    6    0            3            2   
13      10   67     

In [35]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.1, random_state=17)

In [39]:
regressors = [
    LinearRegression(), 
    RANSACRegressor(), 
    KNeighborsRegressor(),
    KNeighborsRegressor(n_neighbors=9, metric='manhattan'),
    SVR(),
    LinearSVR(),
    GaussianProcessRegressor(),
    SVR(kernel='linear'), # Cf. LinearSVR: much slower, might be better or worse: 
]

In [43]:

head = 6
for model in regressors[:head]:
    start = time()
    model.fit(X_train, y_train)
    train_time = time() - start
    start = time()
    predictions = model.predict(X_test)
    predict_time = time()-start    
    print(model)
    print("\tTraining time: %0.3fs)" % train_time)
    print("\tPrediction time: %0.3fs" % predict_time)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
	Training time: 0.004s)
	Prediction time: 0.001s
	Explained variance: 0.2617233150129805
	Mean absolute error: 21.053629248570044
	R2 score: 0.2588572686093088

RANSACRegressor(base_estimator=None, is_data_valid=None, is_model_valid=None,
        loss='absolute_loss', max_skips=inf, max_trials=100,
        min_samples=None, random_state=None, residual_threshold=None,
        stop_n_inliers=inf, stop_probability=0.99, stop_score=inf)
	Training time: 0.080s)
	Prediction time: 0.001s
	Explained variance: -2.2223950898342673
	Mean absolute error: 44.03645731369311
	R2 score: -3.3602221578231477

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=None, n_neighbors=5, p=2,
          weights='uniform')
	Training time: 0.001s)
	Prediction time: 0.002s
	Explained variance: 0.23876125635750223
	Mean absolute error: 21.819354838709682
	R2 score: 0.2223

D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [45]:
lr = LinearRegression()
lasso = Lasso()
ridge = Ridge()

for model in [lr, lasso, ridge]:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(model)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
	Explained variance: 0.2617233150129805
	Mean absolute error: 21.053629248570044
	R2 score: 0.2588572686093088

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)
	Explained variance: 0.2670900679656003
	Mean absolute error: 21.604186414655665
	R2 score: 0.26279836839698656

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)
	Explained variance: 0.26235856258562495
	Mean absolute error: 21.055164798726356
	R2 score: 0.25950857236626657



In [47]:
regressors = [LinearRegression(),
              DecisionTreeRegressor(max_depth=5),
              DecisionTreeRegressor(max_depth=10),
              DecisionTreeRegressor(max_depth=20),
              RandomForestRegressor(max_depth=10),
              SVR(),]

for model in regressors:
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    print(model)
    print("\tExplained variance:", explained_variance_score(y_test, predictions))
    print("\tMean absolute error:", mean_absolute_error(y_test, predictions))
    print("\tR2 score:", r2_score(y_test, predictions))
    print()

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)
	Explained variance: 0.2617233150129805
	Mean absolute error: 21.053629248570044
	R2 score: 0.2588572686093088

DecisionTreeRegressor(criterion='mse', max_depth=5, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
	Explained variance: 0.8375725480277652
	Mean absolute error: 8.231275770070337
	R2 score: 0.833732680233352

DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')
	Explained variance: 0.6921937628930535
	Mean absolute error: 10.118

D:\Anaconda\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
D:\Anaconda\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':
D:\Anaconda\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Anaconda\lib\site-packages\sklearn\svm\base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarnin